In [72]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [73]:
nfet_01v8_lvt = lk('../techsweep/simulation/nfet_01v8_lvt.mat')

#### Specifications

In [74]:
gm = 10e-3

#### Design choices

In [75]:
l = 0.15
gm_id = np.array([6, 20])

#### Sizing and benchmarking

In [76]:
id = gm/gm_id
jd = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id, L=l)
w = id/jd

cgg_w = nfet_01v8_lvt.lookup('CGG_W', GM_ID=gm_id, L=l)
cgg = w*cgg_w
ft = gm/cgg/2/np.pi
cfringe = w*0.239289e-15
cgg_ngspice = cgg-2*cfringe

df = pd.DataFrame([gm_id, id, jd, w, cgg, ft, cgg_ngspice], ['gm_id', 'id', 'jd', 'w', 'cgg', 'ft', 'cgg_ngspice'], columns=['option1', 'option2']); df

,option1,option2
gm_id,6.000000e+00,2.000000e+01
id,1.666667e-03,5.000000e-04
jd,7.100132e-05,3.077393e-06
w,2.347374e+01,1.624752e+02
cgg,2.844918e-14,1.790295e-13
ft,5.594359e+10,8.889872e+09
cgg_ngspice,1.721516e-14,1.012724e-13


#### Spice validation

In [77]:
%%writefile ./sizing_diffpair.spice
** differential pair sizing example

* old binned models
.lib /foss/pdks/sky130A/libs.tech/ngspice/sky130.lib.spice tt
* new continuous models
*.lib /foss/pdks/sky130A/libs.tech/combined/sky130.lib.spice tt

.param mc_mm_switch=0
*.param lx=0.15 wx=23.5 nfx=5 idx=1.6666m
.param lx=0.15 wx=162.5 nfx=40 idx=0.5m
.save @m.xm1a.msky130_fd_pr__nfet_01v8_lvt

XM1a d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM1b d g s 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
vg  g  0  1
vd  d  0  1
is  s  0  {2*idx}

.control
  op
  *show
  print @m.xm1a.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.xm1a.msky130_fd_pr__nfet_01v8_lvt[cgg]
.endc
.end

Overwriting ./sizing_diffpair.spice


In [78]:
!PDK=sky130A /foss/tools/bin/ngspice -b ./sizing_diffpair.spice


Note: Compatibility modes selected: hs a



Circuit: ** differential pair sizing example

option SCALE: Scale is set to 1e-06 for instance and model parameters
Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver
 Reference value :  0.00000e+00
No. of Data Rows : 1
@m.xm1a.msky130_fd_pr__nfet_01v8_lvt[gm] = 1.027211e-02
@m.xm1a.msky130_fd_pr__nfet_01v8_lvt[cgg] = 9.989792e-14
Note: Simulation executed from .control section 
